<a href="https://colab.research.google.com/github/Kwanikaze/vpandas/blob/master/Data_Generator_OHE_block_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
import math
from scipy.linalg import block_diag

In [ ]:
#Just run first 3 cells
card_A = 25
card_B = card_A
card_C = card_A
num_blocks = 2 #number of latent types in each count matrice

In [ ]:
def create_block_diagonal_matrix(card_1,num_blocks): 
  #Generate block matrix for groundtruth AB counts
  #Create smaller square block matrices of constants to be placed into AB block diagonal matrix
  #Determine dimensions of each square block
  blocks_dim = math.floor(card_1 / num_blocks)
  blocks_dim_list = [blocks_dim]*num_blocks
  print(blocks_dim_list)
  #Modify dimension of first block so sum of block dimensions equals card_1
  blocks_dim_list[0] += (card_1 - sum(blocks_dim_list))
  #Create blocks with same constant counts
  blocks = []
  for dim in blocks_dim_list:
    blocks.append(np.ones((dim,dim))*random.randint(3, 5))
  return block_diag(*blocks)
  #print(blocks_dim_list)
  #print(blocks)

def counts_table(card_1,card_2,num_blocks, convertToProb, convertToCond):
  counts_matrix = create_block_diagonal_matrix(card_1,num_blocks)
  if convertToCond:
    #counts_matrix = counts_matrix/counts_matrix.sum(axis=1, keepdims=True)
    counts_matrix = (counts_matrix.T/counts_matrix.sum(axis=1)).T
  #Create columns  
  col1 = []
  col2 = []
  for i in range(0,card_1):
    col1.extend([i]*card_2)
    for j in range(0,card_2):
      col2.extend([j])
  #print(col1)
  #print(col2)
  col_counts = counts_matrix.flatten()
  if convertToProb and not convertToCond:
    col_counts /= col_counts.sum() #normalize counts to probabilities
  #return col_counts
  table = np.array([col1,col2,col_counts],dtype=object).T
  #print(table)
  return table

In [ ]:
#Just run first 3 cells
# Method 2 P(A,B)*P(C|B)
AB_prob = counts_table(card_A,card_B,num_blocks,convertToProb=True,convertToCond=False)
prob_CgivenB = counts_table(card_B,card_C,num_blocks,convertToProb=True,convertToCond=True)
#print(prob_CgivenB)

#CgivenB = np.array([colB,colC,prob_CgivenB],dtype=object).T
#print(BC)
##Generate P(B,C)
prob_AB = pd.DataFrame(data = AB_prob,columns=['A','B','P(A,B)'])
prob_CgivenB =  pd.DataFrame(data = prob_CgivenB,columns=['B','C','P(C|B)'])

#print(prob_AB)
#print(prob_CgivenB)
#print(np.sum(prob_CgivenB))

#Merge prob_AB and prob_CgivenB
merged = prob_AB.merge(prob_CgivenB, on=['B'])
merged['P(A,B,C)'] = merged['P(A,B)'] * merged['P(C|B)']
result = merged
result = merged.drop(['P(A,B)', 'P(C|B)'], axis=1)
print(result)
print(merged['P(A,B,C)'].sum())

#Convert probabilities to counts
result['P(A,B,C)'] = result['P(A,B,C)']*5*card_A*card_B*card_C
print(result)
global_population = []
newdf = pd.DataFrame(np.repeat(result.values,result['P(A,B,C)'],axis=0))
newdf.columns = result.columns
print('the min number ' + str(min(newdf['P(A,B,C)'])))
del newdf['P(A,B,C)']
print(newdf.shape)
file_name = 'data_'+str(card_A) + '.csv'
newdf.to_csv(file_name,index=False)
print(newdf)


[12, 12]
[12, 12]
        A   B   C     P(A,B,C)
0       0   0   0  0.000270665
1       0   0   1  0.000270665
2       0   0   2  0.000270665
3       0   0   3  0.000270665
4       0   0   4  0.000270665
...    ..  ..  ..          ...
15620  24  24  20  0.000234577
15621  24  24  21  0.000234577
15622  24  24  22  0.000234577
15623  24  24  23  0.000234577
15624  24  24  24  0.000234577

[15625 rows x 4 columns]
1.0000000000000424
        A   B   C P(A,B,C)
0       0   0   0  21.1457
1       0   0   1  21.1457
2       0   0   2  21.1457
3       0   0   3  21.1457
4       0   0   4  21.1457
...    ..  ..  ..      ...
15620  24  24  20  18.3263
15621  24  24  21  18.3263
15622  24  24  22  18.3263
15623  24  24  23  18.3263
15624  24  24  24  18.3263

[15625 rows x 4 columns]
the min number 18.32629603565564
(77241, 3)
        A   B   C
0       0   0   0
1       0   0   0
2       0   0   0
3       0   0   0
4       0   0   0
...    ..  ..  ..
77236  24  24  24
77237  24  24  24
77238  24

In [ ]:
#Create truth table columns for A and C
colB = []
colC = []
for i in range(0,card_B):
  colB.extend([i]*card_B)
  for j in range(0,card_C):
    colC.extend([j])
print(colB)
print(colC)

#Generate random values that sum to 1 for P(A,B), but tweak one value to be favourable
s = np.random.random(card_B*card_C)
#print(s[1])
#s[1] += 50/(card_B*card_C)
#print(s[1])
s /= s.sum()
print(s)

#Obtain P(C|B) from P(B,C) / P(B)
prob_CgivenB = []
for i in range(0,card_B*card_C):
  Bval = math.floor(i/card_B)
  #print(Bval)
  #print(s[i])
  #print(s[5*Bval:(5*(Bval+1))])
  val = np.sum(s[5*Bval:(5*(Bval+1))])
  adjusted_val = s[i] / val
  prob_CgivenB.append(adjusted_val) 
print(prob_CgivenB)
print(np.sum(prob_CgivenB))
# Create P(A,B,C) from P(AB) and P(BC)

CgivenB = np.array([colB,colC,prob_CgivenB],dtype=object).T
#print(BC)
##Generate P(B,C)
prob_CgivenB =  pd.DataFrame(data = CgivenB,columns=['B','C','P(C|B)'])
print(prob_CgivenB)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 1

In [ ]:
#Merge prob_AB and prob_CgivenB
merged = prob_AB.merge(prob_CgivenB, on=['B'])
merged['P(A,B,C)'] = merged['P(A,B)'] * merged['P(C|B)']
result = merged
result = merged.drop(['P(A,B)', 'P(C|B)'], axis=1)
print(result)
print(merged['P(A,B,C)'].sum())

       A   B   C     P(A,B,C)
0      0   0   0  0.000800555
1      0   0   1   0.00116339
2      0   0   2  0.000120229
3      0   0   3  0.000114399
4      0   0   4  0.000841971
...   ..  ..  ..          ...
7995  19  19  15  0.000391907
7996  19  19  16  0.000534159
7997  19  19  17  0.000711989
7998  19  19  18   0.00069543
7999  19  19  19   0.00061436

[8000 rows x 4 columns]
4.28947898243629


In [ ]:
result['P(A,B,C)'] = result['P(A,B,C)']*30*card_A*card_B*card_C
print(result)
global_population = []


       A   B   C P(A,B,C)
0      0   0   0  192.133
1      0   0   1  279.214
2      0   0   2  28.8549
3      0   0   3  27.4558
4      0   0   4  202.073
...   ..  ..  ..      ...
7995  19  19  15  94.0578
7996  19  19  16  128.198
7997  19  19  17  170.877
7998  19  19  18  166.903
7999  19  19  19  147.446

[8000 rows x 4 columns]


In [ ]:
newdf = pd.DataFrame(np.repeat(result.values,result['P(A,B,C)'],axis=0))
newdf.columns = result.columns
del newdf['P(A,B,C)']
print(newdf.shape)

(1025485, 3)


In [ ]:
#newdf.to_csv('data_20.csv',index=False)

In [ ]:
# Method 1 Counts(A,B)*Counts(B,C) - DOES NOT WORK ppandas gives probabilities P(C|A)>0 where groundtruth P(C|A)= 0
AB_counts = count_table(card_A,card_B,num_blocks,convertToProb=False)
BC_counts = count_table(card_B,card_C,num_blocks,convertToProb=False)
AB_df =  pd.DataFrame(data = AB_counts,columns=['A','B','Counts(A,B)'])
BC_df =  pd.DataFrame(data = BC_counts,columns=['B','C','Counts(B,C)'])
merged = AB_df.merge(BC_df, on=['B'])
print(merged.shape)
merged['Counts(A,B,C)'] = merged['Counts(A,B)'] * merged['Counts(B,C)']
merged.drop(['Counts(A,B)', 'Counts(B,C)'], axis=1, inplace=True)
#print(merged)
#print(merged['Counts(A,B,C)'].sum())
newdf = pd.DataFrame(np.repeat(merged.values,merged['Counts(A,B,C)'],axis=0))
newdf.columns = result.columns
del newdf['Counts(A,B,C)']
print(newdf.shape)
print(newdf)
file_name = 'data_'+str(card_A) + '.csv'
newdf.to_csv(file_name,index=False)
#print(prob_AB)
#print(AB_counts)
#print(BC_counts)


ValueError: ignored